In [ ]:
''' numpy - типов данных
    pandas - чтения csv 
    wave - чтение wave файла
    os - проверка существования пути файла
'''

import numpy as np
import pandas as pd
import wave
import os


# Типы данных амплитуд
types = {
    1: np.int8,
    2: np.int16,
    4: np.int32,
}


# Открытие файла на разбор
def wave_open(filepath):
    if not os.path.exists(filepath):
        return None
    with open(filepath, 'r') as file_handler:
        wavf =  wave.open(filepath, mode='r')
        return wavf

''' Путь к файлу '''
wav = wave_open("/Users/nikita/Documents/python_3_filies/ML/ML_sound_1/output_vot_e.wav")

# Различные вкусности из wave-файла
''' Кол-во каналов '''
def nchannals():
    nchannals_var = wav.getnchannels()
    return nchannals_var

''' Длина семпла (int8, int16, int32) '''
def sampwidth():
    sampwidth_var = wav.getsampwidth()
    return  sampwidth_var

''' Частота окон '''
def framerate():
    framerate_var = wav.getframerate()
    return framerate_var

''' Кол-во окон '''
def nframes():
    nframes_var = wav.getnframes()
    return  nframes_var

''' Тип сжатия '''
def comptype():
    comptype_var = wav.getcomptype()
    return comptype_var

''' Название сжатия '''
def compname():
    compname_var = wav.getcompname()
    return compname_var

''' Длительность записи '''
def duration():
    duration_var = nframes()/framerate()
    return duration_var

# Значения амплитуд в 10-ричной системе счисления
def channel_data():
    byte_content = wav.readframes(nframes())
    samples = np.fromstring(byte_content, dtype=types[sampwidth()])
    #for n in range(nchannals()):
    channel = samples
    return channel

''' Битовые данные'''
byte_content = wav.readframes(nframes())

''' Представление битов в int8(16,32) '''
samples = np.fromstring(byte_content, dtype=types[sampwidth()])

''' Образка первых 1000 семплов, т.к. там ненужный шум, позже можно будет убрать '''
samples = samples[1000:]
#samples = samples[131000:132000]

''' Логарифмирование семплов + минимальное значение int16 (т.к. запись была в int16), ибо логарифм не может иметь отрицательный аргумент '''
samples_log = np.log(samples+32768)
len(samples)

In [ ]:
''' График оригинальных семплов '''
plt.plot(range(len(samples)),samples)
plt.show()

In [ ]:
''' График прологарифмированных семплов '''
plt.plot(range(len(samples_log)),samples_log)
plt.show()

In [ ]:
''' Преобразование Фурье из прологарифмированных семплов '''
samples_fourier = np.fft.fft(samples_log)

In [ ]:
''' Действительная часть пребразования фурье '''
plt.plot(range(len(samples_fourier[1:])),samples_fourier.real[1:],)#'ro')
plt.show()

In [ ]:
import pyaudio
import wave

''' Параметры wave файла '''
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 3

''' Путь к файлу '''
WAVE_OUTPUT_FILENAME = "/Users/nikita/Documents/python_3_filies/ML/ML_sound_1/output_vot_e.wav"

''' Класс подключения к портам аудио '''
p = pyaudio.PyAudio()

''' Иницилиазыция открытия с параметрами '''
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

''' Завершение записи '''
stream.stop_stream()
stream.close()
p.terminate()

''' Запись в файл '''
wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()